In [28]:
import sys
import os

# Adiciona o caminho da pasta 'scripts' ao sys.path
sys.path.append(os.path.abspath('../scripts'))

Lendo os dados da tabela Gold das Empresa A e B e gravando no Dataframe

In [ ]:
# Importação das bibliotecas
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import time
import urllib
import pandas as pd
from db import *

engine = get_postgres_engine()
engine2 = get_sqlserver_engine()

df = pd.read_sql('SELECT * FROM "TB_VENDAS_EMPRESA_B_GOLD"', engine)
df2 = pd.read_sql('SELECT * FROM TB_VENDAS_EMPRESA_A_GOLD', engine2)

# Exibe os primeiros registros
print(df.head())
print(df2.head())

Padronizando os nomes das colunas

In [ ]:
df.columns = ['NomeProduto', 'CategoriaProduto', 'DataVenda', 'EstadoFilial', 'ValorTotalVenda']
df2.columns = ['NomeProduto', 'CategoriaProduto', 'DataVenda', 'EstadoFilial', 'ValorTotalVenda']
print(df.head())
print(df2.head())

Feito a união dos dataframes e gravado na tabela

In [53]:
df_final = pd.concat([df, df2], ignore_index=True)
print(len(df_final))

4401


In [ ]:
df_final['ValorTotalVenda'] = df_final['ValorTotalVenda'].astype('string').apply(lambda x: format(float(x), '.2f'))
df_final.drop_duplicates(inplace=True)
print(df_final)

In [ ]:
# Envia os dados para o Postgres
df_final.to_sql("TB_VENDAS_EMPRESA_A_EMPRESA_B", engine2, if_exists="replace", index=False)


Fazendo requisição da API dos dados para o enriquecimento

In [41]:
import requests
import json

def carregar_dados_api(url):

    try:
        response = requests.get(url)
        response.raise_for_status()  # dispara erro se status != 200

        dados_json = response.json()

        # se for um JSON diretamente em forma de lista de dicionários:
        dfj = pd.DataFrame(dados_json['value'])

        return dfj

    except Exception as e:
        print(f"Erro ao acessar a API: {e}")
        return None

In [ ]:
url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2020'&@dataFinalCotacao='07-15-2025'&$top=1400&$format=json"

df_api = carregar_dados_api(url)  # se os dados estiverem em 'resultados'

if df_api is not None:
    print(df_api.head())

In [43]:
df_api['dataHoraCotacao'] = pd.to_datetime(df_api['dataHoraCotacao'], errors='coerce')

Tratamentos nos dados da API e gravado em uma tabela auxiliar

In [ ]:
# Extrai apenas a data
df_api['DataCotacao'] = df_api['dataHoraCotacao'].dt.date

# Pega a última ocorrência de hora para cada data
df_ultima_ocorrencia = df_api.sort_values('dataHoraCotacao').groupby('DataCotacao').tail(1).reset_index(drop=True)

print(df_ultima_ocorrencia)

In [45]:
print(len(df_ultima_ocorrencia))

1388


In [46]:
df_api.columns = ['CotacaoCompra', 'CotacaoVenda', 'DataHoraCotacao', 'DataCotacao']

In [48]:
# Envia os dados para o SQL Server
df_api.to_sql("TB_VENDAS_AUXILAR", engine2, if_exists="replace", index=False)

print("✅ Dados carregados com sucesso no SQL Server!")

✅ Dados carregados com sucesso no SQL Server!
